In [41]:
import cv2 , time,pandas
from datetime import datetime

In [ ]:
first_frame = None
status_list = [None,None]
times = []
df = pandas.DataFrame(columns=['start','end']) 

video = cv2.VideoCapture(0)

while video:
    check , res= video.read()
    if check==False:
        break
    #res = cv2.resize(frame, (frame.shape[0]//2,frame.shape[1]//2), interpolation = cv2.INTER_LINEAR)
    status = 0
    gray = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(21,21),0)
    
    if first_frame is None:
        first_frame = gray 
        continue
    
    delta_frame = cv2.absdiff(first_frame , gray)
    threshold_delta = cv2.threshold(delta_frame,30,255,cv2.THRESH_BINARY)[1]
    threshold_delta = cv2.dilate(threshold_delta,None,iterations=0)
    cnts,_ = cv2.findContours(threshold_delta.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    for contour  in cnts :
        if cv2.contourArea(contour)<1000:
            continue
        status = 1
        (x,y,w,h) = cv2.boundingRect(contour)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
    status_list.append(status)
    status_list = status_list[-2:]
    
    if status_list[-1]==1 and status_list[-2]==0:
        times.append(datetime.now())
    if status_list[-1]==0 and status_list[-2]==1:
        times.append(datetime.now())
        
    cv2.imshow('frame',res)
    cv2.imshow('gray_scale',gray)
    cv2.imshow('delta',delta_frame)
    cv2.imshow('threshold',threshold_delta)
    
    #key = cv2.waitKey(key)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
print(status_list)
print(times)
print(len(times))  

for i in range(0,len(times)-1,2):
        df = df.append({"start":times[i],"end":times[i+1]},ignore_index=True)
df.to_csv("Times.csv")


video.release()
cv2.destroyAllWindows()

In [39]:
from bokeh.plotting import figure,show,output_file
from bokeh.models import HoverTool ,ColumnDataSource

In [40]:
if len(times)!=1:
    df["start_string"] = df["start"].dt.strftime("%Y-%m-%d %H:%M:%S")
    df["end_string"] = df["end"].dt.strftime("%Y-%m-%d %H:%M:%S")

    cds = ColumnDataSource(df)

    p=figure(x_axis_type='datetime',height=100, width=500,title= "Motion Graph")
    p.yaxis.minor_tick_line_color = None
    #p.ygrid[0].ticker.desired_num_ticks = 1
    p.yaxis.ticker.desired_num_ticks = 1

    hover = HoverTool(tooltips=[("start","@start_string"),("end","@end_string")])
    p.add_tools(hover)

    q = p.quad(left="start",right="end",bottom=0,top=1,color="red",source=cds)

    output_file("Graph1.html")
    show(p)